In [ ]:
import sklearn
import pandas as pd
import numpy as np
import nltk
import sklearn
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import matplotlib
from sklearn.model_selection import cross_val_score
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn.model_selection import train_test_split
import re
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
import pickle

In [ ]:
smoking_ex1 = pickle.load(open('../best_models/best_smoking_ex1.pkl', 'rb'))
smoking_ex2 = pickle.load(open('../best_models/best_smoking_ex2.pkl', 'rb'))

drinking_ex1 = pickle.load(open('../best_models/best_drinking_ex1.pkl', 'rb'))
drinking_ex2 = pickle.load(open('../best_models/best_drinking_ex2.pkl', 'rb'))

drugs_ex1 = pickle.load(open('../best_models/best_drugs_ex1.pkl', 'rb'))
drugs_ex2 = pickle.load(open('../best_models/best_drugs_ex2.pkl', 'rb'))

In [ ]:
stemmer = SnowballStemmer("dutch")
np.random.seed(500)

def create_preprocess_corpus(input_csv, content_name, label_name, skip_rows=[]):
    corpus = pd.read_csv(input_csv,encoding='latin-1', skiprows=skip_rows)
    corpus = corpus.rename({content_name:'text', label_name: 'label'}, axis=1)
    corpus['text'] = corpus['text'].str.replace('\t',' ')
    corpus.drop(['Unnamed: 0'], axis=1, inplace=True)
    corpus.drop_duplicates(inplace=True)
    corpus['text'] = corpus['text'].astype(str)
    corpus['text'] = corpus['text'].str.lower()
    #corpus['text'].replace(to_replace="[^\w\s]", value="", regex=True, inplace=True)
    corpus['text'] = [stemmer.stem(text) for text in corpus['text']]
    corpus = corpus.drop(corpus[corpus.label == '--'].index)
    corpus_backup = corpus.copy()
    return corpus, corpus_backup

In [ ]:
full_smoking_corpus, full_smoking_corpus_backup = create_preprocess_corpus("../../input_data/smoking/labelled_smoking.csv", "roken_report_content", "roken_answer_label")

In [ ]:
full_drinking_corpus, full_drinking_corpus_backup = create_preprocess_corpus("../../input_data/drinking/labelled_drinking.csv", "alcohol_report_content", "alcohol_answer_label", skip_rows=[227225])

In [ ]:
full_drugs_corpus, full_drugs_corpus_backup = create_preprocess_corpus("../../input_data/drugs/labelled_drugs.csv", "drugs_report_content", "drugs_answer_label")

In [ ]:
_, smoking_x_test, _, smoking_y_test = train_test_split(full_smoking_corpus['text'], full_smoking_corpus['label'], test_size=0.2, random_state=50)

In [ ]:
_, drinking_x_test, _, drinking_y_test = train_test_split(full_drinking_corpus['text'], full_drinking_corpus['label'], test_size=0.2, random_state=50)

In [ ]:
_, drugs_x_test, _, drugs_y_test = train_test_split(full_drugs_corpus['text'], full_drugs_corpus['label'], test_size=0.2, random_state=50)

In [ ]:
predicted_nb = smoking_ex1.predict(smoking_x_test)
print(np.mean(predicted_nb == smoking_y_test))
cm = confusion_matrix(smoking_y_test, predicted_nb)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=smoking_ex1.classes_)
disp.plot()

plt.show()
print(metrics.classification_report(smoking_y_test, predicted_nb,
    target_names=smoking_ex1.classes_))

In [ ]:
ex1_features = smoking_ex1['tfidf'].get_feature_names_out()
ex1_coef = smoking_ex1['clf'].best_estimator_.coef_

In [ ]:
ex1_features

In [ ]:
feature_importance = pd.DataFrame(ex1_features, columns=['feature'])

In [ ]:
feature_importance['nothing_found_importance'] = ex1_coef[0]
feature_importance['current_user_importance'] = ex1_coef[1]
feature_importance['non_user_importance'] = ex1_coef[2]
feature_importance['previous_user_importance'] = ex1_coef[3]

In [ ]:
ex1_coef.shape

In [ ]:
smoking_ex1['clf'].best_estimator_.classes_

In [ ]:
feature_importance

In [ ]:
feature_importance.sort_values(by='non_user_importance', ascending=False)